In [30]:
import matplotlib.pyplot as plt  # Veri görselleştirme
import pandas as pd  # Veri işleme
import os  # Dosya işlemleri
import librosa  # Ses analizi
import numpy as np  # Sayısal işlemler
from tqdm import tqdm  # Progres bar
from sklearn.model_selection import train_test_split  # Veri seti ayırma
from tensorflow.keras.utils import to_categorical  # Etiketleri kategorik hale getirme
from sklearn.preprocessing import LabelEncoder  # Etiketleri sayısallaştırma
from tensorflow.keras.models import Sequential  # Model yapısı
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten  # Katmanlar
from tensorflow.keras.optimizers import Adam  # Optimizasyon
from sklearn import metrics  # Model değerlendirme
from tensorflow.keras.callbacks import ModelCheckpoint  # Callback fonksiyonu
from datetime import datetime  # Tarih ve saat
import warnings  # Uyarılar
warnings.filterwarnings('ignore')  # Uyarıları kapatma

In [31]:
# UrbanSound8K veri setindeki ses dosyaları ve metadata'yı yükleyip, metadata'nın ilk 5 satırını görüntüleme
audio_dataset_path='UrbanSound8K/audio/'
metadata=pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [3]:
# Bu fonksiyon, ses dosyasından MFCC özelliklerini çıkarır ve ortalamalarını alarak döndürme
def features_extractor(filename):
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast')  
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)  
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)  

    return mfccs_scaled_features  

In [4]:
# Ses dosyalarından özellikleri çıkarıp etiketlerle birlikte listeye ekleme
extracted_features = []  
for index_num, row in tqdm(metadata.iterrows()):  
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))  
    final_class_labels = row["class"]  
    data = features_extractor(file_name)  
    extracted_features.append([data, final_class_labels])  

8732it [10:31, 13.82it/s]


In [5]:
# Çıkarılan özellikleri ve etiketleri bir DataFrame'e dönüştürme
extracted_features_df = pd.DataFrame(extracted_features, columns=['feature', 'class'])  
extracted_features_df.head()  

,feature,class
0,"[-214.95764, 70.502464, -130.70279, -53.116936...",dog_bark
1,"[-423.7311, 109.2299, -52.872597, 60.827538, 0...",children_playing
2,"[-458.52844, 121.35432, -46.535675, 51.96947, ...",children_playing
3,"[-413.63254, 101.61351, -35.43868, 53.047153, ...",children_playing
4,"[-446.38693, 113.68634, -52.457195, 60.349735,...",children_playing


In [6]:
# Özellikler (X) ve etiketler (y) numpy dizilerine dönüştürme
X = np.array(extracted_features_df['feature'].tolist())  
y = np.array(extracted_features_df['class'].tolist())  

In [7]:
# Etiketleri sayısal hale getirir ve one-hot encoding uygular
labelencoder = LabelEncoder()  
y = to_categorical(labelencoder.fit_transform(y))  

In [8]:
# Eğitim ve test verilerini %80-%20 oranında ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
# x eğitim verisinin şekli (boyutları) yazdıralım
X_train.shape

(6985, 40)

In [10]:
# x test verisinin şekli (boyutları) yazdıralım
X_test.shape

(1747, 40)

In [11]:
# y eğitim verisinin şekli (boyutları) yazdıralım
y_train.shape

(6985, 10)

In [12]:
# y test verisinin şekli (boyutları) yazdıralım
y_test.shape

(1747, 10)

In [13]:
# Etiketlerin sayısını belirler (sınıf sayısı)
num_labels = 10

In [14]:
# Modelin katmanlarını tanımlama
model = Sequential()  

# İlk gizli katman
model.add(Dense(125, input_shape=(40,)))  
model.add(Activation('relu'))  
model.add(Dropout(0.5))  

# İkinci gizli katman
model.add(Dense(200))  
model.add(Activation('relu'))  
model.add(Dropout(0.5))  

# Üçüncü gizli katman
model.add(Dense(125))  
model.add(Activation('relu'))  
model.add(Dropout(0.5))  

# Çıktı katmanı
model.add(Dense(num_labels))  
model.add(Activation('softmax'))  

In [15]:
# Modelin özetini yazdırma, katmanlar ve parametreler hakkında bilgi verme
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 125)               5125      
_________________________________________________________________
activation (Activation)      (None, 125)               0         
_________________________________________________________________
dropout (Dropout)            (None, 125)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               25200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 125)               2

In [16]:
# Modeli derler, kayıp fonksiyonu, metrik ve optimizasyon algoritmasını belirleme
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [17]:
# Modeli eğitim verisi ile eğitme, doğrulama verisiyle test etme
epochscount = 400  
num_batch_size = 32  

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=epochscount, validation_data=(X_test, y_test), verbose=1)

Epoch 1/400
219/219 [==============================] - 0s 2ms/step - loss: 10.8966 - accuracy: 0.1294 - val_loss: 2.2738 - val_accuracy: 0.1397
Epoch 2/400
219/219 [==============================] - 0s 2ms/step - loss: 2.6575 - accuracy: 0.1427 - val_loss: 2.2689 - val_accuracy: 0.1380
Epoch 3/400
219/219 [==============================] - 0s 2ms/step - loss: 2.3225 - accuracy: 0.1482 - val_loss: 2.1989 - val_accuracy: 0.1431
Epoch 4/400
219/219 [==============================] - 0s 2ms/step - loss: 2.2067 - accuracy: 0.1931 - val_loss: 2.0926 - val_accuracy: 0.2152
Epoch 5/400
219/219 [==============================] - 0s 2ms/step - loss: 2.1249 - accuracy: 0.2145 - val_loss: 1.9930 - val_accuracy: 0.2959
Epoch 6/400
219/219 [==============================] - 0s 2ms/step - loss: 2.0443 - accuracy: 0.2510 - val_loss: 1.8933 - val_accuracy: 0.3572
Epoch 7/400
219/219 [==============================] - 0s 2ms/step - loss: 1.9785 - accuracy: 0.2646 - val_loss: 1.8129 - val_accuracy: 0.337

Epoch 58/400
219/219 [==============================] - 0s 2ms/step - loss: 0.9779 - accuracy: 0.6683 - val_loss: 0.7861 - val_accuracy: 0.7459
Epoch 59/400
219/219 [==============================] - 1s 2ms/step - loss: 0.9815 - accuracy: 0.6669 - val_loss: 0.8059 - val_accuracy: 0.7521
Epoch 60/400
219/219 [==============================] - 0s 2ms/step - loss: 0.9935 - accuracy: 0.6628 - val_loss: 0.8042 - val_accuracy: 0.7453
Epoch 61/400
219/219 [==============================] - 0s 2ms/step - loss: 0.9740 - accuracy: 0.6729 - val_loss: 0.7981 - val_accuracy: 0.7407
Epoch 62/400
219/219 [==============================] - 0s 2ms/step - loss: 0.9904 - accuracy: 0.6704 - val_loss: 0.7781 - val_accuracy: 0.7527
Epoch 63/400
219/219 [==============================] - 0s 2ms/step - loss: 0.9505 - accuracy: 0.6772 - val_loss: 0.7794 - val_accuracy: 0.7441
Epoch 64/400
219/219 [==============================] - 0s 2ms/step - loss: 0.9564 - accuracy: 0.6782 - val_loss: 0.7617 - val_accuracy:

Epoch 115/400
219/219 [==============================] - 0s 2ms/step - loss: 0.8718 - accuracy: 0.7144 - val_loss: 0.7131 - val_accuracy: 0.7819
Epoch 116/400
219/219 [==============================] - 0s 2ms/step - loss: 0.8575 - accuracy: 0.7160 - val_loss: 0.6779 - val_accuracy: 0.7934
Epoch 117/400
219/219 [==============================] - 0s 2ms/step - loss: 0.8504 - accuracy: 0.7144 - val_loss: 0.6853 - val_accuracy: 0.7779
Epoch 118/400
219/219 [==============================] - 0s 2ms/step - loss: 0.8533 - accuracy: 0.7097 - val_loss: 0.6692 - val_accuracy: 0.7951
Epoch 119/400
219/219 [==============================] - 0s 2ms/step - loss: 0.8657 - accuracy: 0.7130 - val_loss: 0.6914 - val_accuracy: 0.7882
Epoch 120/400
219/219 [==============================] - 0s 2ms/step - loss: 0.8563 - accuracy: 0.7174 - val_loss: 0.6904 - val_accuracy: 0.7871
Epoch 121/400
219/219 [==============================] - 0s 2ms/step - loss: 0.8573 - accuracy: 0.7099 - val_loss: 0.6793 - val_ac

219/219 [==============================] - 0s 2ms/step - loss: 0.8123 - accuracy: 0.7317 - val_loss: 0.6582 - val_accuracy: 0.7997
Epoch 172/400
219/219 [==============================] - 0s 2ms/step - loss: 0.8261 - accuracy: 0.7244 - val_loss: 0.6435 - val_accuracy: 0.8060
Epoch 173/400
219/219 [==============================] - 1s 2ms/step - loss: 0.8008 - accuracy: 0.7331 - val_loss: 0.6442 - val_accuracy: 0.8008
Epoch 174/400
219/219 [==============================] - 1s 2ms/step - loss: 0.7880 - accuracy: 0.7346 - val_loss: 0.6432 - val_accuracy: 0.7997
Epoch 175/400
219/219 [==============================] - 1s 2ms/step - loss: 0.8006 - accuracy: 0.7380 - val_loss: 0.6667 - val_accuracy: 0.7899
Epoch 176/400
219/219 [==============================] - 1s 2ms/step - loss: 0.8158 - accuracy: 0.7241 - val_loss: 0.6547 - val_accuracy: 0.8019
Epoch 177/400
219/219 [==============================] - 0s 2ms/step - loss: 0.8152 - accuracy: 0.7323 - val_loss: 0.6593 - val_accuracy: 0.7962

219/219 [==============================] - 1s 3ms/step - loss: 0.7638 - accuracy: 0.7508 - val_loss: 0.6215 - val_accuracy: 0.8060
Epoch 228/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7880 - accuracy: 0.7429 - val_loss: 0.6248 - val_accuracy: 0.7997
Epoch 229/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7870 - accuracy: 0.7402 - val_loss: 0.6245 - val_accuracy: 0.8157
Epoch 230/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7822 - accuracy: 0.7337 - val_loss: 0.6035 - val_accuracy: 0.8163
Epoch 231/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7915 - accuracy: 0.7366 - val_loss: 0.6369 - val_accuracy: 0.8065
Epoch 232/400
219/219 [==============================] - 1s 3ms/step - loss: 0.8025 - accuracy: 0.7266 - val_loss: 0.6192 - val_accuracy: 0.8151
Epoch 233/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7696 - accuracy: 0.7486 - val_loss: 0.6206 - val_accuracy: 0.8071

219/219 [==============================] - 1s 3ms/step - loss: 0.7714 - accuracy: 0.7453 - val_loss: 0.6202 - val_accuracy: 0.8151
Epoch 284/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7526 - accuracy: 0.7446 - val_loss: 0.6297 - val_accuracy: 0.8168
Epoch 285/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7511 - accuracy: 0.7486 - val_loss: 0.6324 - val_accuracy: 0.8100
Epoch 286/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7628 - accuracy: 0.7457 - val_loss: 0.6392 - val_accuracy: 0.8140
Epoch 287/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7931 - accuracy: 0.7396 - val_loss: 0.6237 - val_accuracy: 0.8111
Epoch 288/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7575 - accuracy: 0.7509 - val_loss: 0.6001 - val_accuracy: 0.8220
Epoch 289/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7687 - accuracy: 0.7476 - val_loss: 0.6139 - val_accuracy: 0.8060

219/219 [==============================] - 1s 3ms/step - loss: 0.7487 - accuracy: 0.7585 - val_loss: 0.5944 - val_accuracy: 0.8174
Epoch 340/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7329 - accuracy: 0.7582 - val_loss: 0.6086 - val_accuracy: 0.8077
Epoch 341/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7586 - accuracy: 0.7516 - val_loss: 0.6147 - val_accuracy: 0.8122
Epoch 342/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7404 - accuracy: 0.7536 - val_loss: 0.6036 - val_accuracy: 0.8237
Epoch 343/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7551 - accuracy: 0.7533 - val_loss: 0.6091 - val_accuracy: 0.8094
Epoch 344/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7386 - accuracy: 0.7459 - val_loss: 0.5930 - val_accuracy: 0.8145
Epoch 345/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7589 - accuracy: 0.7509 - val_loss: 0.6073 - val_accuracy: 0.8180

219/219 [==============================] - 1s 3ms/step - loss: 0.7415 - accuracy: 0.7549 - val_loss: 0.6089 - val_accuracy: 0.8088
Epoch 396/400
219/219 [==============================] - 1s 2ms/step - loss: 0.7426 - accuracy: 0.7559 - val_loss: 0.5810 - val_accuracy: 0.8266
Epoch 397/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7333 - accuracy: 0.7569 - val_loss: 0.5861 - val_accuracy: 0.8208
Epoch 398/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7306 - accuracy: 0.7573 - val_loss: 0.5758 - val_accuracy: 0.8288
Epoch 399/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7272 - accuracy: 0.7601 - val_loss: 0.5937 - val_accuracy: 0.8254
Epoch 400/400
219/219 [==============================] - 1s 3ms/step - loss: 0.7311 - accuracy: 0.7565 - val_loss: 0.6024 - val_accuracy: 0.8088


In [18]:
# Modelin test verisi üzerindeki doğruluğunu değerlendirme ve yazdırma
validation_test_set_accuracy = model.evaluate(X_test, y_test, verbose=0)  
print(validation_test_set_accuracy[1])

0.8088151216506958


In [19]:
# Test setindeki ilk örneği yazdırma
X_test[1]

array([-465.89066   ,    1.1322267 ,  -34.01122   ,   35.365334  ,
        -14.865485  ,  -19.09527   ,   -0.5655371 ,  -16.102211  ,
        -21.32794   ,    7.7042246 ,  -29.161444  ,  -18.939268  ,
         -2.9505708 ,   -8.135996  ,  -15.148013  ,   -6.580668  ,
         -7.555405  ,    9.359482  ,   14.432987  ,   21.94775   ,
         20.870613  ,    1.3638178 ,  -19.213827  ,   -4.613979  ,
         -1.0521431 ,    3.2284064 ,   -6.983857  ,  -16.403254  ,
        -10.018602  ,   13.024475  ,    0.5268234 ,  -23.8531    ,
        -15.273722  ,    9.262864  ,   10.387702  ,   -0.56245726,
         -1.2470247 ,   17.716091  ,   13.853949  ,   -5.1873612 ],
      dtype=float32)

In [20]:
# Test verisi üzerinde tahmin yapar ve sınıf etiketlerine dönüştürür
predictions = model.predict(X_test)  
predicted_classes = np.argmax(predictions, axis=1)  # En yüksek olasılığa sahip sınıfı seçer

In [21]:
# İnternetten indirdiğim PoliceSiren ses dosyasını yükledim, MFCC özelliklerini çıkarttım ve ortalama ölçeklenmiş özellikleri hesaplattım
filename="UrbanSound8K/PoliceSiren.wav"  
sound_signal, sample_rate = librosa.load(filename, res_type='kaiser_fast')  
mfccs_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=40)  
mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

In [22]:
# MFCC özelliklerinin ölçeklenmiş ortalamasını yazdırma
print(mfccs_scaled_features)

[-5.0994394e+02  5.8154579e+01 -2.6709690e+01 -1.7371210e+01
 -2.8767126e+00  1.1947034e+01  3.4051540e+01  2.4762865e+01
  4.6231513e+00  2.6400737e-03  4.9524341e+00  1.8476988e+00
 -1.0729708e+01 -1.1758752e+01 -9.5040531e+00 -1.4617859e+01
 -1.1907148e+01 -7.1361742e+00 -6.3340759e+00 -2.0190914e-01
  1.8497976e+00 -4.4656420e+00 -6.6696696e+00 -4.1695404e+00
 -8.3585262e-01  1.0366113e+00 -1.6953439e+00 -3.1226525e+00
  6.9590539e-01  4.1818194e+00  6.1561000e-01 -7.5990481e+00
 -7.3445063e+00 -1.6433215e+00 -1.6265788e+00 -1.0085020e+00
 -7.9554778e-01 -3.7891550e+00 -2.6074238e+00 -2.7007205e+00]


In [23]:
# MFCC özelliklerini 1 satır ve -1 sütun olacak şekilde şekilledirme
mfccs_scaled_features = mfccs_scaled_features.reshape(1, -1)

In [24]:
# MFCC özelliklerinin şekillendirilmiş halinin boyutunu yazdırma
print(mfccs_scaled_features.shape)

(1, 40)


In [25]:
# MFCC özelliklerinin şekillendirilmiş halini yazdırma
print(mfccs_scaled_features)

[[-5.0994394e+02  5.8154579e+01 -2.6709690e+01 -1.7371210e+01
  -2.8767126e+00  1.1947034e+01  3.4051540e+01  2.4762865e+01
   4.6231513e+00  2.6400737e-03  4.9524341e+00  1.8476988e+00
  -1.0729708e+01 -1.1758752e+01 -9.5040531e+00 -1.4617859e+01
  -1.1907148e+01 -7.1361742e+00 -6.3340759e+00 -2.0190914e-01
   1.8497976e+00 -4.4656420e+00 -6.6696696e+00 -4.1695404e+00
  -8.3585262e-01  1.0366113e+00 -1.6953439e+00 -3.1226525e+00
   6.9590539e-01  4.1818194e+00  6.1561000e-01 -7.5990481e+00
  -7.3445063e+00 -1.6433215e+00 -1.6265788e+00 -1.0085020e+00
  -7.9554778e-01 -3.7891550e+00 -2.6074238e+00 -2.7007205e+00]]


In [27]:
# Modelin tahmin sonucunu alma
result_array = model.predict(mfccs_scaled_features)  

In [28]:
# Tahmin sonucunu yazdırma
print(result_array)

[[6.2269909e-32 4.2608523e-15 1.4426780e-12 1.3274234e-06 3.5735236e-12
  2.9260118e-17 3.8060243e-14 4.5421141e-29 9.9999869e-01 1.0312334e-17]]


In [29]:
# Tahmin edilen sınıfı yazdırma
result_classes = ["air_conditioner","car_horn","children_playing","dog_bark","drilling", 
                  "engine_idling", "gun_shot", "jackhammer", "siren", "street_music"]

result = np.argmax(result_array[0])  # En yüksek olasılığa sahip sınıfı bulur
print(result_classes[result])  # Tahmin edilen sınıfı yazdırır

siren
